In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

Модель teoogherghi/Log-Analysis-Model-DistilBert — это fine-tuned версия distilbert-base-uncased, обученная для анализа логов.
Ссылка: https://huggingface.co/teoogherghi/Log-Analysis-Model-DistilBert

Также есть модель от пользователя Slavka, fine-tuned bert-base-cased для парсинга логов Windows:
https://huggingface.co/Slavka/bert-base-cased-finetuned-log-parser-winlogbeat_nowhitespace

https://par.nsf.gov/servlets/purl/10317798

https://www.researchgate.net/publication/354743341_LogBERT_Log_Anomaly_Detection_via_BERT

https://dr.ntu.edu.sg/bitstream/10356/181426/2/mathematics-12-02758-v2.pdf

In [ ]:
model_name = "teoogherghi/Log-Analysis-Model-DistilBert" 

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def embed_logs(log_texts):
    inputs = tokenizer(log_texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:,0,:].cpu().numpy()
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True) + 1e-10
    return embeddings

In [ ]:
logs = [
    "ERROR: failed to connect to database",
    "INFO: user login successful",
    "WARN: disk space low",
]

In [ ]:
embeddings = embed_logs(logs)
print(embeddings.shape)

Детали для аномалий с LogBERT

    Обучаешь модель на «нормальных» логах (masked LM).

    Во время инференса считаешь вероятность (loss) предсказания токенов.

    Логи с низкой вероятностью — аномальные.

    Можно строить порог по loss и детектировать аномалии.